In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/CUT')

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python test.py --dataroot /content/drive/MyDrive/ --name base202111252 --CUT_mode CUT --phase train --direction AtoB --dataset_mode aligned 


----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 4                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/       	[default: placeholder]
             dataset_mode: aligned                       	[default: unaligned]
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: latest                        
                     eval: False                         
        flip_equivariance: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 3                

### 首先先进入输出文件夹
在输出文件夹里执行下面的程序
注意修改读取文件目录和输出文件目录

In [ ]:
import os
os.chdir('/content/drive/MyDrive/CUT/results/base202111252/train_latest/images/fake_B')

In [ ]:
files = os.listdir('/content/drive/MyDrive/CUT/results/base202111252/train_latest/images/fake_B')

In [ ]:
print(len(files))

12288


In [ ]:
# from types import prepare_class
import numpy as np
from glob import glob
import torch
import os
from PIL import Image

COL = 8 #指定拼接图片的列数
ROW = 8 #指定拼接图片的行数
UNIT_HEIGHT_SIZE = 256 #图片高度
UNIT_WIDTH_SIZE = 256 #图片宽度
SAVE_QUALITY = 100 #保存的图片的质量 可选0-100
def concat_images(image_names, name, savePath):
    image_files = []
    for i in range(COL):
        for j in range(ROW):

            image_files.append(Image.open(image_names[i][j]))

    target = Image.new('RGB', (UNIT_WIDTH_SIZE * COL, UNIT_HEIGHT_SIZE * ROW)) #创建成品图的画布
    #第一个参数RGB表示创建RGB彩色图，第二个参数传入元组指定图片大小，第三个参数可指定颜色，默认为黑色
    for row in range(ROW):
        for col in range(COL):
            #对图片进行逐行拼接
            #paste方法第一个参数指定需要拼接的图片，第二个参数为二元元组（指定复制位置的左上角坐标）
            #或四元元组（指定复制位置的左上角和右下角坐标）
            target.paste(image_files[COL*row+col], (0 + UNIT_WIDTH_SIZE*col, 0 + UNIT_HEIGHT_SIZE*row))
    target.save(savePath+'/' + name + '.jpg', quality=SAVE_QUALITY) #成品图保存

files=np.loadtxt('/content/drive/MyDrive/testlist.txt',dtype=str)

for i in files:
    tensorsum = torch.tensor([])
    hfile = []

    if i.split('_')[0]!= '18':

        continue
    for k in range(8):
        print(i)
        sig = i +'_' + str(k) 

        hfile.append(glob(sig+'*'))

    savePath = '/content/drive/MyDrive/OUTPUT2/18'
    concat_images(hfile,i,savePath)

### 可以加上颜色提取功能
如果加上颜色提取功能
需要先进去OUTPUT文件夹下
最好还是直接全部下载下来吧


In [ ]:
from skimage.color import rgb2lab, deltaE_cie76, deltaE_ciede2000

In [1]:
!python --version

Python 3.7.12


In [ ]:
import skimage
print(skimage.__version__)

0.18.3


In [ ]:
import numpy as np
from numpy.lib.function_base import blackman
from numpy.lib.npyio import save
from skimage import io
from skimage.color import rgb2lab, deltaE_cie76, deltaE_ciede2000
import os
from glob import glob

def caculate(folder,save_path):
    files = os.listdir(folder)
    files.sort()
    # print(files)
    acc_arr = []
    
    for file in files:
        path = os.path.join(folder,file)
        rgb = io.imread(path)
        # print(file)
        

        green = [113,79,77]
        magenta = [220,20,60]
        white = [255,255,255]
        gray2 = [142,142,140]
        gray = [81,80,85]
        black = [5,5,7]
        pink = [161,65,162]
        black2 = [51,50,46]
        threshold_green = 25

        green_3d = np.uint8(np.asarray([[green]]))
        black_3d = np.uint8(np.asarray([[black]]))
        gray_3d = np.uint8(np.asarray([[gray]]))
        pink_3d = np.uint8(np.asarray([[pink]]))
        black2_3d = np.uint8(np.asarray([[black2]]))


        lab = rgb2lab(rgb)
        dE_green = deltaE_cie76(np.squeeze(rgb2lab(green_3d)), np.squeeze(lab))
        

        matrix = dE_green < threshold_green

        num1 = np.where(matrix==True)


        acc = len(num1[0])/(4194304)

        acc_arr.append(acc)
        

        rgb[dE_green < threshold_green] = magenta

        save_path_img = os.path.join(save_path,file)
        io.imsave(save_path_img,rgb)

    return acc_arr


save_path = '/content/drive/MyDrive/outfake'

Acc = caculate('/content/drive/MyDrive/OUTPUT2/18',save_path)
np.savetxt('/content/drive/MyDrive/outfake/out18fakenew.txt',Acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_0_0.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_0_3.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_0_7.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_1_0.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_1_7.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_2_0.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: /content/drive/MyDrive/outfake/18_2_7.jpg is a low contras

In [ ]:
save_path = '/content/drive/MyDrive/outfake'

Acc = caculate('/content/drive/MyDrive/OUTPUT2/18',save_path)
np.savetxt('/content/drive/MyDrive/outfake/out18fake.txt',Acc)

In [ ]:
save_path = '/content/drive/MyDrive/outfake'

Acc = caculate('/content/drive/MyDrive/OUTPUT2/19',save_path)
np.savetxt('/content/drive/MyDrive/outfake/out19fake.txt',Acc)

### 绘制直方图